# Test embedalign with SentEval 

This notebook will allow you to test EmbedAlign using SentEval. In particular, this also works on **CPUs** :D

* Dependencies:
    * Python 3.5 with NumPy/SciPy
    * Pytorch 
    * Tensorflow 1.5.0  (for CPUs or GPUs depending on how you plan to run it)
        * For example in MacOS: 
        ```
        pip install https://storage.googleapis.com/tensorflow/mac/cpu/tensorflow-1.5.0-py3-none-any.whl
        ```
    * scikit-learn>=0.18.0
    * dill>=0.2.7.1


* Install `dgm4nlp` by following the instructions [here](https://github.com/uva-slpl/dgm4nlp), we highly recommend the use of `virtualenv`.

In the same `virtualenv`, do the following:

* Clone repo from FAIR github
```
    git clone https://github.com/facebookresearch/SentEval.git
    cd SentEval/
```

* Install senteval
```
    python setup.py install
```

* Download datasets (it takes some time...)
    * these are downstream tasks
    * new Senteval also has probing tasks (https://github.com/facebookresearch/SentEval/tree/master/data/probing) for evaluating linguistic properties of your embeddings. 

```
    cd data/downstream/
    ./get_transfer_data.bash
```

* Download [pretained embedlaign model](https://surfdrive.surf.nl/files/index.php/s/9M4h5zqmYETSmf3)


* The following code evaluates embedalign pretrained embeddings on en-fr Europarl on different NLP downstream tasks.



In [7]:
from __future__ import absolute_import, division, unicode_literals

import sys
import numpy as np
import logging
import sklearn
import os
import data
import pprint

# data.py is part of Senteval and it is used for loading word2vec style files
# Set PATHs
# path to senteval
PATH_TO_SENTEVAL = 'SentEval'
# path to the NLP datasets 
PATH_TO_DATA = os.path.join('SentEval', 'data')

# import SentEval
sys.path.insert(0, PATH_TO_SENTEVAL)
import senteval

import tensorflow as tf
import logging
from collections import defaultdict
import dill
import dgm4nlp

In [3]:
class dotdict(dict):
    """ dot.notation access to dictionary attributes """
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

class EmbeddingExtractor:
    """
    This will compute a forward pass with the inference model of EmbedAlign and 
        give you the variational mean for each L1 word in the batch.
        
    Note that this takes monolingual L1 sentences only (at this point we have a traiend EmbedAlign model
        which dispenses with L2 sentences).    
        
    You don't really want to touch anything in this class.
    """

    def __init__(self, graph_file, ckpt_path, config=None):        
        g1 = tf.Graph()
        self.meta_graph = graph_file
        self.ckpt_path = ckpt_path
        
        self.softmax_approximation = 'botev-batch' #default
        with g1.as_default():
            self.sess = tf.Session(config=config, graph=g1)
            # load architecture computational graph
            self.new_saver = tf.train.import_meta_graph(self.meta_graph)
            # restore checkpoint
            self.new_saver.restore(self.sess, self.ckpt_path) #tf.train.latest_checkpoint(
            self.graph = g1  #tf.get_default_graph()
            # retrieve input variable
            self.x = self.graph.get_tensor_by_name("X:0")
            # retrieve training switch variable (True:trianing, False:Test)
            self.training_phase = self.graph.get_tensor_by_name("training_phase:0")
            #self.keep_prob = self.graph.get_tensor_by_name("keep_prob:0")

    def get_z_embedding_batch(self, x_batch):
        """
        :param x_batch: is np array of shape [batch_size, longest_sentence] containing the unique ids of words
        
        :returns: [batch_size, longest_sentence, z_dim]        
        """
        # Retrieve embeddings from latent variable Z
        # we can sempale several n_samples, default 1
        try:
            z_mean = self.graph.get_tensor_by_name("z:0")
            
            feed_dict = {
                self.x: x_batch,
                self.training_phase: False,
                #self.keep_prob: 1.

            }
            z_rep_values = self.sess.run(z_mean, feed_dict=feed_dict) 
        except:
            raise ValueError('tensor Z not in graph!')
        return z_rep_values

## Load global unigram dict

In [28]:
with open('europarl_unigram.pickle', 'rb') as f:
    unigram = dill.load(f)

## Different types of combination functions

In [46]:
def simple_mean_batcher(params, batch):
    """
    At this point batch is a python list containing sentences. Each sentence is a list of tokens (each token a string).
    The code below will take care of converting this to unique ids that EmbedAlign can understand.
    
    This function should return a single vector representation per sentence in the batch.
    In this example we use the average of word embeddings (as predicted by EmbedAlign) as a sentence representation.
    
    In this method you can do mini-batching or you can process sentences 1 at a time (batches of size 1).
    We choose to do it 1 sentence at a time to avoid having to deal with masking. 
    
    This should not be too slow, and it also saves memory.
    """
    # if a sentence is empty dot is set to be the only token
    # you can change it into NULL dependening in your model
    batch = [sent if sent != [] else ['.'] for sent in batch]
    embeddings = []
    for sent in batch:
        # Here is where dgm4nlp converts strings to unique ids respecting the vocabulary
        # of the pre-trained EmbedAlign model
        # from tokens ot ids position 0 is en
        x1 = params.tks1[0].to_sequences([(' '.join(sent))])
        
        # extract word embeddings in context for a sentence
        # [1, sentence_length, z_dim]
        z_batch1 = params.extractor.get_z_embedding_batch(x_batch=x1)
        
        # Sentence embedding is the average of the words in a sentence
        # [1, z_dim]
        sent_vec = np.mean(batch, axis=1)
        
        # check if there is any NaN in vector (they appear sometimes when there's padding)
        if np.isnan(sent_vec.sum()):
            sent_vec = np.nan_to_num(sent_vec)        
        embeddings.append(sent_vec)
    embeddings = np.vstack(embeddings)
    return embeddings

def unigram_weighted_batcher(params, batch):
    """
    At this point batch is a python list containing sentences. Each sentence is a list of tokens (each token a string).
    The code below will take care of converting this to unique ids that EmbedAlign can understand.
    
    This function should return a single vector representation per sentence in the batch.
    In this example we use the average of word embeddings (as predicted by EmbedAlign) as a sentence representation.
    
    In this method you can do mini-batching or you can process sentences 1 at a time (batches of size 1).
    We choose to do it 1 sentence at a time to avoid having to deal with masking. 
    
    This should not be too slow, and it also saves memory.
    """
    
    # Hyperparameter value taken from https://openreview.net/pdf?id=SyK00v5xx
    a = 10e-3
    
    # if a sentence is empty dot is set to be the only token
    # you can change it into NULL dependening in your model
    batch = [sent if sent != [] else ['.'] for sent in batch]
    embeddings = []
    for sent in batch:
        # Here is where dgm4nlp converts strings to unique ids respecting the vocabulary
        # of the pre-trained EmbedAlign model
        # from tokens ot ids position 0 is en
        x1 = params.tks1[0].to_sequences([(' '.join(sent))])
        
        # extract word embeddings in context for a sentence
        # [1, sentence_length, z_dim]
        z_batch1 = params.extractor.get_z_embedding_batch(x_batch=x1)
                
        # Gather the weights for this sentence (https://openreview.net/pdf?id=SyK00v5xx)
        weights = [a / (a + unigram[word.lower()]) for word in sent]
        
        # every sequence starts with 2, which is the id for -NULL- (assuming that is padding)
        # so it gets a weight of 0
        weights = [0] + weights
        weights = np.array(weights)
        
        # Add dimensions to weight array to match shape of z_batch1
        weights = weights.reshape(-1, z_batch1.shape[1], 1)
        
        # Sentence embedding is the average of the words in a sentence
        # [1, z_dim]
        sent_vec = np.mean(weights * z_batch1, 1)
        
        # check if there is any NaN in vector (they appear sometimes when there's padding)
        if np.isnan(sent_vec.sum()):
            sent_vec = np.nan_to_num(sent_vec)        
        embeddings.append(sent_vec)
    embeddings = np.vstack(embeddings)
    return embeddings

This is how you interface with SentEval. The only think you need to change are the paths to trained models in the main block at the end.

In [48]:
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#



# Set PATHs
# path to senteval
#PATH_TO_SENTEVAL = '../'



# import SentEval
#sys.path.insert(0, PATH_TO_SENTEVAL)

# Set params for SentEval
# we use logistic regression (usepytorch: Fasle) and kfold 10
# In this dictionary you can add extra information that you model needs for initialization
# for example the path to a dictionary of indices, of hyper parameters
# this dictionary is passed to the batched and the prepare fucntions
params_senteval = {'task_path': '',
                   'usepytorch': False,
                   'kfold': 10,
                   'ckpt_path': '',
                   'tok_path': '',
                   'extractor': None,
                   'tks1': None}
# made dictionary a dotdict
params_senteval = dotdict(params_senteval)
# this is the config for the NN classifier but we are going to use scikit-learn logistic regression with 10 kfold
# usepytorch = False 
#params_senteval['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
#                                 'tenacity': 3, 'epoch_size': 2}


def prepare(params, samples):
    """
    In this example we are going to load a tensorflow model, 
    we open a dictionary with the indices of tokens and the computation graph
    """
    params.extractor = EmbeddingExtractor(
        graph_file='%s.meta'%(params.ckpt_path),
        ckpt_path=params.ckpt_path,
        config=None #run in cpu
    )

    # load tokenizer from training
    params.tks1 = dill.load(open(params.tok_path, 'rb'))
    return


# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

if __name__ == "__main__":
    # define paths
    # path to senteval data
    # note senteval adds downstream into the path
    params_senteval.task_path = PATH_TO_DATA 
    # path to computation graph
    # we use best model on validation AER

    params_senteval.ckpt_path = os.path.join('ull-practical3-embedalign', 'model.best.validation.aer.ckpt')
    
    # path to tokenizer with ids of trained Europarl data
    # out dictionary id depends on dill for pickle
    params_senteval.tok_path = os.path.join('ull-practical3-embedalign', 'tokenizer.pickle')
    # we use 10 fold cross validation
    params_senteval.kfold = 10
    se = senteval.engine.SE(params_senteval, unigram_weighted_batcher, prepare)
    
    # here you define the NLP taks that your embedding model is going to be evaluated
    # in (https://arxiv.org/abs/1802.05883) we use the following :
    # SICKRelatedness (Sick-R) needs torch cuda to work (even when using logistic regression), 
    # but STS14 (semantic textual similarity) is a similar type of semantic task
    transfer_tasks = ['MR', 'CR', 'MPQA', 'SUBJ', 'SST2', 'TREC',
                      'MRPC', 'SICKEntailment']

    # senteval prints the results and returns a dictionary with the scores
    results = se.eval(transfer_tasks)
    pprint.pprint(results)

2018-05-21 16:26:03,920 : ***** Transfer task : MR *****




INFO:tensorflow:Restoring parameters from ull-practical3-embedalign/model.best.validation.aer.ckpt


2018-05-21 16:26:06,963 : Restoring parameters from ull-practical3-embedalign/model.best.validation.aer.ckpt
2018-05-21 16:26:07,783 : Generating sentence embeddings
2018-05-21 16:26:08,886 : Generated sentence embeddings
2018-05-21 16:26:08,887 : Training sklearn-LogReg with (inner) 10-fold cross-validation
2018-05-21 16:26:08,981 : Best param found at split 1: l2reg = 8                 with score 59.24
2018-05-21 16:26:09,110 : Best param found at split 2: l2reg = 0.25                 with score 57.71
2018-05-21 16:26:09,199 : Best param found at split 3: l2reg = 1                 with score 59.71
2018-05-21 16:26:09,289 : Best param found at split 4: l2reg = 0.25                 with score 58.38
2018-05-21 16:26:09,380 : Best param found at split 5: l2reg = 0.25                 with score 58.38
2018-05-21 16:26:09,471 : Best param found at split 6: l2reg = 2                 with score 59.67
2018-05-21 16:26:09,564 : Best param found at split 7: l2reg = 0.25                 with scor

INFO:tensorflow:Restoring parameters from ull-practical3-embedalign/model.best.validation.aer.ckpt


2018-05-21 16:26:13,086 : Restoring parameters from ull-practical3-embedalign/model.best.validation.aer.ckpt
2018-05-21 16:26:14,183 : Generating sentence embeddings
2018-05-21 16:27:04,887 : Generated sentence embeddings
2018-05-21 16:27:04,889 : Training sklearn-LogReg with (inner) 10-fold cross-validation
2018-05-21 16:27:09,936 : Best param found at split 1: l2reg = 8                 with score 70.62
2018-05-21 16:27:14,726 : Best param found at split 2: l2reg = 4                 with score 69.92
2018-05-21 16:27:19,206 : Best param found at split 3: l2reg = 2                 with score 70.33
2018-05-21 16:27:23,893 : Best param found at split 4: l2reg = 8                 with score 70.24
2018-05-21 16:27:28,628 : Best param found at split 5: l2reg = 8                 with score 70.18
2018-05-21 16:27:33,208 : Best param found at split 6: l2reg = 8                 with score 70.47
2018-05-21 16:27:37,911 : Best param found at split 7: l2reg = 8                 with score 70.06
2018

...

INFO:tensorflow:Restoring parameters from ull-practical3-embedalign/model.best.validation.aer.ckpt


2018-05-21 16:47:32,076 : Restoring parameters from ull-practical3-embedalign/model.best.validation.aer.ckpt
2018-05-21 16:47:33,441 : Computing embedding for test
2018-05-21 16:48:23,334 : Computed test embeddings
2018-05-21 16:48:23,335 : Computing embedding for train
2018-05-21 16:50:31,351 : Computed train embeddings
2018-05-21 16:50:31,377 : Training sklearn-LogReg with 10-fold cross-validation
2018-05-21 16:50:43,778 : [('reg:0.5', 70.66), ('reg:1', 70.76), ('reg:2', 70.86), ('reg:4', 70.54), ('reg:8', 70.56), ('reg:16', 70.66), ('reg:32', 70.61)]
2018-05-21 16:50:43,780 : Cross-validation : best param found is reg = 2             with score 70.86
2018-05-21 16:50:43,781 : Evaluating...
2018-05-21 16:50:43,939 : Dev acc : 70.86 Test acc 71.25; Test F1 80.58 for MRPC.

2018-05-21 16:50:43,946 : ***** Transfer task : SICK-Entailment*****




INFO:tensorflow:Restoring parameters from ull-practical3-embedalign/model.best.validation.aer.ckpt


2018-05-21 16:50:47,494 : Restoring parameters from ull-practical3-embedalign/model.best.validation.aer.ckpt
2018-05-21 16:50:53,606 : Computing embedding for test
2018-05-21 16:52:04,637 : Computed test embeddings
2018-05-21 16:52:04,639 : Computing embedding for train
2018-05-21 16:53:10,655 : Computed train embeddings
2018-05-21 16:53:10,657 : Computing embedding for dev
2018-05-21 16:53:18,077 : Computed dev embeddings
2018-05-21 16:53:18,113 : Training sklearn-LogReg with standard validation..
2018-05-21 16:53:21,499 : [('reg:0.25', 71.0), ('reg:0.5', 71.6), ('reg:1', 72.0), ('reg:2', 71.0), ('reg:4', 70.6), ('reg:8', 72.0)]
2018-05-21 16:53:21,501 : Validation : best param found is reg = 1 with score             72.0
2018-05-21 16:53:21,502 : Evaluating...
2018-05-21 16:53:21,916 : 
Dev acc : 72.0 Test acc : 72.97 for                        SICK entailment



{'CR': {'acc': 70.01, 'devacc': 70.2, 'ndev': 3775, 'ntest': 3775},
 'MPQA': {'acc': 84.07, 'devacc': 83.97, 'ndev': 10606, 'ntest': 10606},
 'MR': {'acc': 49.32, 'devacc': 58.95, 'ndev': 74, 'ntest': 74},
 'MRPC': {'acc': 71.25,
          'devacc': 70.86,
          'f1': 80.58,
          'ndev': 4076,
          'ntest': 1725},
 'SICKEntailment': {'acc': 72.97, 'devacc': 72.0, 'ndev': 500, 'ntest': 4927},
 'SST2': {'acc': 68.04, 'devacc': 67.78, 'ndev': 872, 'ntest': 1821},
 'SUBJ': {'acc': 99.6, 'devacc': 99.6, 'ndev': 5020, 'ntest': 5020},
 'TREC': {'acc': 56.4, 'devacc': 52.82, 'ndev': 5452, 'ntest': 500}}


In [24]:
import pprint

pprint.pprint(results)

{'CR': {'acc': 70.65, 'devacc': 70.64, 'ndev': 3775, 'ntest': 3775},
 'MPQA': {'acc': 83.78, 'devacc': 83.89, 'ndev': 10606, 'ntest': 10606},
 'MR': {'acc': 56.94, 'devacc': 58.5, 'ndev': 74, 'ntest': 74},
 'MRPC': {'acc': 70.96,
          'devacc': 70.64,
          'f1': 80.1,
          'ndev': 4076,
          'ntest': 1725},
 'SICKEntailment': {'acc': 74.75, 'devacc': 72.6, 'ndev': 500, 'ntest': 4927},
 'SST2': {'acc': 67.05, 'devacc': 67.2, 'ndev': 872, 'ntest': 1821},
 'SUBJ': {'acc': 99.6, 'devacc': 99.6, 'ndev': 5020, 'ntest': 5020},
 'TREC': {'acc': 56.8, 'devacc': 53.39, 'ndev': 5452, 'ntest': 500}}
